In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

import sys
sys.path.append('../')

import nn.traintestNNs as traintestNNs
import nn.symmequiv as symmequiv
from nn.lib import symmpartitions

In [2]:
import importlib
importlib.reload(symmequiv)
importlib.reload(symmpartitions)

<module 'nn.lib.symmpartitions' from '../nn/lib/symmpartitions.py'>

In [3]:
#device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [4]:
device = 'cpu'

In [5]:
print(device)

cpu


In [6]:
symm_model = symmequiv.SymmetricGrpEquivLinear(dim_n = 5, order_k = 3, order_l = 3).to(device)

In [7]:
symm_model.num_weights

202

In [8]:
symm_model.basis_set_matrices[0]

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [9]:
class SymmetricGrpEquivNN(nn.Module):
  def __init__(self, dim_n):
    super().__init__()
    self.n = dim_n
    self.layer1 = nn.Sequential(
        symmequiv.SymmetricGrpEquivLinear(dim_n = self.n, order_k = 3, order_l = 2).to(device),
        nn.ReLU(),
        symmequiv.SymmetricGrpEquivLinear(dim_n = self.n, order_k = 2, order_l = 2).to(device),
        nn.ReLU(),
        symmequiv.SymmetricGrpEquivLinear(dim_n = self.n, order_k = 2, order_l = 0).to(device)
    )

  def forward(self, x):
    x = self.layer1(x)
    return x

In [10]:
symm = SymmetricGrpEquivNN(dim_n = 9).to(device)

In [11]:
len(list(symm.parameters())) - 3

69